# 📢📢📢 Projet NLP : La parole aux citoyens (Make.org - Le tourisme vert en Ille-et-Vilaine)📢📢📢
# Représentation des mots et des phrases
## Bibliothèques nécessaires

In [1]:
import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
import plotly.offline as pyo
from gensim.models import Word2Vec
import numpy as np
from typing import Tuple
import ast

## Lecture des données

In [2]:
df = pd.read_csv(
    "../data/processed/req_tourisme_responsable_tokenised.csv",
    delimiter=";",
    encoding="utf-8"
)

df.head(3)

,content,agree_count,disagree_count,neutral_count,agree_score,disagree_score,neutral_score,doNotCare,doNotUnderstand,doable,...,platitudeAgree,platitudeDisagree,nb_interrogations,nb_exclamation,tokens,case,tokens_normalized,stop_words,tokens_final,tokens_lemmatized
0,Il faut une surveillance des lieux touristique...,27,7,7,0.65,0.18,0.17,1,0,8,...,3,4,0,0,"['Il', 'faut', 'une', 'surveillance', 'des', '...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['une', 'surveillance', 'des', 'lieux', 'touri...","['il', 'une', 'des', 'avec', 'la', 'pour', 'ou']","['surveillance', 'lieux', 'amende', 'clef', 'd...","['surveillance', 'lieux', 'touristique', 'amen..."
1,Il faut donner des sacs poubelles et des cendr...,56,18,15,0.62,0.19,0.19,0,0,21,...,5,7,0,0,"['Il', 'faut', 'donner', 'des', 'sacs', 'poube...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['donner', 'des', 'sacs', 'poubelles', 'et', '...","['il', 'des', 'et', 'des', 'de', 'aux', 'des',...","['donner', 'sac', 'poubelle', 'cendrier', 'poc...","['donner', 'sac', 'poubelle', 'cendrier', 'poc..."
2,Il faut récompenser et mettre en avant les act...,35,7,14,0.62,0.13,0.25,0,5,12,...,1,5,0,0,"['Il', 'faut', 'récompenser', 'et', 'mettre', ...","['Min', 'Min', 'Min', 'Min', 'Min', 'Min', 'Mi...","['recompenser', 'et', 'mettre', 'en', 'avant',...","['il', 'et', 'en', 'les', 'des', 'du', 'et', '...","['recompenser', 'mettre', 'avant', 'acte', 'pr...","['recompenser', 'mettre', 'avant', 'acte', 'pr..."


## Chargement des différents modèles

In [3]:
cbow_with_stop_words = Word2Vec.load("../outputs/WE_models/cbow_20D_0")
cbow_without_stop_words = Word2Vec.load("../outputs/WE_models/cbow_20D_1")
skgram_with_stop_words = Word2Vec.load("../outputs/WE_models/skgram_20D_0")
skgram_without_stop_words = Word2Vec.load("../outputs/WE_models/skgram_20D_1")

## Réduction de dimension (2D) et répresentation des mots
### Petite fonction applicable pour chaque modèle

In [4]:
def make_2D_plots(model: Word2Vec, title: str, perplexity: int = 5, metric: str = 'cosine', random_state: int = 42) -> Tuple[pd.DataFrame, px.scatter]:
    """
    Create a 2D plot of word vectors using t-SNE and Plotly.

    Args:
        model (Word2Vec): Trained Word2Vec model.
        title (str): Title for the Plotly chart.
        perplexity (int): Perplexity parameter for t-SNE.
        metric (str): Distance metric for t-SNE ('cosine' recommended).
        random_state (int): Random state for reproducibility.

    Returns:
        Tuple[pd.DataFrame, plotly.graph_objects.Figure]: DataFrame of 2D vectors and the Plotly figure.
    """
    # Get word vectors and vocabulary
    vectors = model.wv.vectors
    words = model.wv.index_to_key

    # Apply t-SNE for dimensionality reduction
    tsne = TSNE(n_components=2, perplexity=perplexity, metric=metric, random_state=random_state)
    vectors_2d = tsne.fit_transform(vectors)

    # Create DataFrame
    df_vectors = pd.DataFrame(vectors_2d, columns=['x', 'y'])
    df_vectors['word'] = words

    # Plot with Plotly
    fig = px.scatter(df_vectors, x='x', y='y', hover_name='word', title=title, color_discrete_sequence=['green'])
    fig.update_traces(textposition='top center', showlegend=False)
    fig.update_xaxes(visible=False)
    fig.update_yaxes(visible=False)
    fig.update_layout(
        template='simple_white',
        title=dict(
            font=dict(size=25, color="black", family="Arial"),
            xanchor="center",
            yanchor="top",
            x=0.5,
            y=0.95
        )
    )

    return df_vectors, fig

In [5]:
figs = {}
for model in [
    (cbow_with_stop_words, "CBOW with Stopwords"),
    (cbow_without_stop_words, "CBOW without Stopwords"),
    (skgram_with_stop_words, "Skip-Gram with Stopwords"),
    (skgram_without_stop_words, "Skip-Gram without Stopwords")
]:
    title = f"2D Word Embeddings ({model[1]})"
    df_2D, fig = make_2D_plots(model[0], title)
    figs[title] = (df_2D, fig)
    pyo.plot(fig, filename=f'../outputs/{title}.html', auto_open=False)

Exception in thread Thread-4 (_readerthread):
Traceback (most recent call last):
  File "c:\Dev\envs\tourisme-vert-env\Lib\threading.py", line 1045, in _bootstrap_inner
    self.run()
  File "c:\Dev\envs\tourisme-vert-env\Lib\site-packages\ipykernel\ipkernel.py", line 766, in run_closure
    _threading_Thread_run(self)
  File "c:\Dev\envs\tourisme-vert-env\Lib\threading.py", line 982, in run
    self._target(*self._args, **self._kwargs)
  File "c:\Dev\envs\tourisme-vert-env\Lib\subprocess.py", line 1599, in _readerthread
    buffer.append(fh.read())
                  ^^^^^^^^^
  File "<frozen codecs>", line 322, in decode
UnicodeDecodeError: 'utf-8' codec can't decode byte 0x82 in position 101: invalid start byte


In [6]:
figs["2D Word Embeddings (Skip-Gram without Stopwords)"][1]

Pour voir les autres représentations, vous disposez de 4 liens permettant d'ouvrir les figures avec votre navigateur web:
- [CBOW sans stopwords](../outputs/2D%20Word%20Embeddings%20(CBOW%20without%20Stopwords).html)
- [CBOW avec stopwords](../outputs/2D%20Word%20Embeddings%20(CBOW%20with%20Stopwords).html)
- [Skip-Gram sans stopwords](../outputs/2D%20Word%20Embeddings%20(Skip-Gram%20without%20Stopwords).html)
- [Skip-Gram avec stopwords](../outputs/2D%20Word%20Embeddings%20(Skip-Gram%20with%20Stopwords).html)

La meilleure représentation pour moi, c'est la version Skip-Gram sans Stopwords. C'est donc ce modèle qui sera séléctionné pour le reste du projet.

## Construction des embedding des propositions
Technique simple et efficace : pour avoir la représentation d'une phrase, on construit le vecteur moyen des mots qu'elle contient. Ici, il s'agira uniquement des tokens.

### Convertion en liste de str

In [7]:
# On réutilise la convertion en liste de str avant d'effectuer un calcul de moyenne
df['tokens_final'] = df['tokens_final'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

### Petite fonction pour calculer la moyenne par ligne

In [8]:
def mean_prop_embedding(tokens:list[str], model:Word2Vec) -> np.array:
    # Initialiser une liste pour stocker les embeddings de chaque token
    embedding_list = []
    
    # Parcourir chaque token dans la liste de tokens
    for token in tokens:
        # Vérifier si le token est présent dans les embeddings du modèle
        if token in model.wv:
            # Ajouter le vecteur d'embedding du token à la liste
            embedding_list.append(model.wv.get_vector(token))
    
    # Si aucun token n'est présent dans les embeddings, retourner un vecteur nul
    if not embedding_list:
        return np.zeros(model.vector_size)
    
    # Calculer la moyenne des embeddings
    mean_embedding = np.mean(embedding_list, axis=0)
    
    # Supprimer la liste des embeddings pour libérer de la mémoire
    del embedding_list
    
    return mean_embedding

In [9]:
df['embedding_prop'] = df['tokens_final'].apply(lambda x : mean_prop_embedding(x, skgram_without_stop_words))

In [10]:
df['embedding_prop'].head(3)

0    [0.026475444, 0.013547266, 0.04194331, -0.1040...
1    [0.03417883, 0.057504207, 0.025103373, -0.1194...
2    [-0.011080465, 0.053166457, 0.0025109972, -0.0...
Name: embedding_prop, dtype: object

### Création d'une matrice pour permettre la réduction de dimension (TSNE)

In [11]:
# Convertir la colonne 'embedding_prop' en une seule matrice
embedding_matrix = np.stack(df['embedding_prop'].values)
embedding_matrix.shape

(1493, 20)

In [12]:
# Réduire les dimensions à deux dimensions avec t-SNE
tsne = TSNE(n_components=2, random_state=42, perplexity=5, metric='cosine')
df[['embedding_prop_x','embedding_prop_y']] = tsne.fit_transform(embedding_matrix)

In [13]:
df[['content','embedding_prop_x','embedding_prop_y']].head(3)

,content,embedding_prop_x,embedding_prop_y
0,Il faut une surveillance des lieux touristique...,60.746189,15.809195
1,Il faut donner des sacs poubelles et des cendr...,54.815662,12.159612
2,Il faut récompenser et mettre en avant les act...,-22.491800,47.501347


In [14]:
# Créer un graphique dynamique avec Plotly
fig = px.scatter(df[['content','embedding_prop_x','embedding_prop_y']], x='embedding_prop_x', y='embedding_prop_y', hover_name='content', color_discrete_sequence=['green'])
fig.update_traces(textposition='top center', showlegend=False)
fig.update_xaxes(visible=False)
fig.update_yaxes(visible=False)
fig.update_layout(
    title=dict(
        text='2D Sentence Embeddings (Skip-Gram without Stopwords)',
        font=dict(size=25, color="black", family="Arial"),
            xanchor="center",
            yanchor="top",
            x=0.5,
            y=0.95
    ),
    template='simple_white',
    hoverlabel=dict(font_size=16),
    )
fig.show()

In [15]:
# Sauvegarder le graphique en tant que fichier HTML
pyo.plot(fig, filename='../outputs/2D Sentence Embeddings (Skip-Gram without Stopwords).html', auto_open=False)

'../outputs/2D Sentence Embeddings (Skip-Gram without Stopwords).html'

## Sauvegarde du dataframe
NB : je n'utilise pas les csv parce que pour les vecteurs c'est pas terrible. Il faut les reconstruire après sauvegarde je ne trouve pas ça pratique

In [16]:
# Convertir tous les vecteurs de la colonne 'embedding_prop' en float64
df['embedding_prop'] = df['embedding_prop'].apply(
    lambda x: np.array(x, dtype=np.float64)
)

In [17]:
# Sauvegarde des jeux de données
df.to_parquet(
    "../data/processed/req_tourisme_responsable_embeddings.parquet",
    engine='pyarrow',
    index=False
)
